# Compound Matrix Method Evans Function

### Note: Define all needed symbols outside the class

In [3]:
from sympy import *
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy import linalg as la
from time import time

In [4]:
from IPython.display import display, Markdown, Latex

In [5]:
plt.rcParams['figure.figsize'] = 15, 5

In [6]:
class SystemSolve():
    def __init__(self,dim,terms,A,deriv,e_val, e_vec,print_on=True, comp=False):
        self.dim = dim
        self.terms = terms
        self.A = A
        self.deriv = deriv
        self._createB()
        self.p = print_on
        self.comp = comp
        self._setEigs(e_val, e_vec)
        return
    
    def _createB(self):
        self.B = []
        for i in range(self.terms):
            b = []
            for j in range(1,self.dim+1):
                b.append(sympify('B{}_{}'.format(i,j)))
            self.B.append(b)
        return
    
    def _setEigs(self, e_val, e_vec):
        self.e_val = e_val
        self.B_tups = []
        b_tups = []
        for i in range(self.dim):
            b_tups.append((self.B[0][i],e_vec[i]))
        
        self.B_tups = b_tups
        #self.B_tups.append(b_tups)
        return
    
    def createSys(self):
        W = []
        W_deriv = []

        for j in range(self.dim):
            w = sympify(0)
            for i in range(self.terms):
                w += self.B[i][j]*sigma**i
            w_deriv = diff(w, sigma)

            W.append(w)
            W_deriv.append(w_deriv)

        # Store W and W' as numpy arrays
        self.W = np.array(W)
        self.W_deriv = np.array(W_deriv)

        # Replace matrix A by A-lam*I
        Anew = self.A - np.eye(self.dim)*self.e_val
        
        # Define system of equations from each row of A
        eqs = []
        for i in range(self.dim):
            eq = np.dot(Anew[i],self.W)-self.W_deriv[i]*self.deriv
            eqs.append(eq)
   
        # Isolate coefficients for each power of sigma
        eq_der = [eq.copy() for eq in eqs]
        heqs = []

        for i in range(self.terms):
            if i==0: prod = 1
            else: 
                prod *= i

            if self.p: print('\n\npower = {}'.format(i)) 
            res = [eqd.subs(sigma,0)/prod for eqd in eq_der]

            if self.p:
                for r in res:
                    print('\n')
                    print(r)

            heqs.append(res)    
            eq_der = [diff(eqd,sigma) for eqd in eq_der]

        # Solve for the coefficients of W
        
        up = 0
        stop = False
        
        # If there are issues uncomment these two lines
        #up = 1
        #stop = True
        
        start = time()
        for j in range(len(heqs)):
            if j == len(heqs)-1 and stop:
                break
            if self.p:
                p1 = '-'*21+'\nsystem when sigma = '
                p2 = str(j)+'\n'+'-'*21+'\n'
                print(p1+p2)
                
            if j == 0:
                try: 
                    matrix, vector = linear_eq_to_matrix(heqs[j],self.B[j+up])
                    soln = matrix.LUsolve(vector)
                    #print('made it here')
                except: 
                    #print('failed')
                    up = 0
                    stop = False
            
            matrix, vector = linear_eq_to_matrix(heqs[j],self.B[j+up])
            
            if self.comp:
                vector = np.array([complex(vector[i].subs(self.B_tups)) for i in range(self.dim)])
                matrix = np.array([[complex(matrix[i,j].subs(self.B_tups)) for j in range(self.dim)] for i in range(self.dim)])
            
            #matrix = np.array(matrix.tolist()).astype(complex)
            #vector = np.array(vector.tolist()).astype(complex)

            if self.comp:
                soln = np.linalg.solve(matrix,vector)
            else:
                soln = matrix.LUsolve(vector)

            for i in range(self.dim):
                if self.comp: 
                    val = soln[i]
                    val = complex(val)
                else:
                    val = soln[i].subs(self.B_tups)
                
                if j != 0 or stop: 
                    self.B_tups.append((self.B[j+up][i],val))
                if self.p: 
                    val = str(self.B[j+up][i]) +' = '+str(val)
                    print(val,'\n\n')

    def evalW(self, sig_val):
        to_sub = self.B_tups + [(sigma,sig_val)]
        
        W_vals = []
        
        for i in range(self.dim):
            val = self.W[i].subs(self.B_tups).subs(sigma,sig_val)
            if self.comp: val = complex(val)
            W_vals.append(val)
        return W_vals
    
    def checkBNorms(self):
        norms = []
        for b in self.B:
            norm = sympify(0)
            for i in b:
                norm = norm + Abs(i)
            norm = (norm)**(1/2)
            
            norm = norm.subs(self.B_tups)
            print(norm)
            norms.append(norm)

        return norms

## Evans Function

In [7]:
# this is for 2 dimension
"""def lift_A(a):
    A_lift = np.array([[a[0,0] + a[1,1], a[1,2], a[1,3], a[1,4], -a[0,2], -a[0,3], -a[0,4], 0, 0, 0],\
             [a[2,1], a[0,0] + a[2,2], a[2,3], a[2,4], a[0,1], 0, 0, -a[0,3], -a[0,4], 0],\
             [a[3,1], a[3,2], a[0,0] + a[3,3], a[3,4], 0, a[0,1], 0, a[0,2], 0, -a[0,4]],\
             [a[4,1], a[4,2], a[4,3], a[0,0] + a[4,4], 0, 0, a[0,1], 0, a[0,2], a[0,3]],\
             [-a[2,0], a[1,0], 0, 0, a[1,1] + a[2,2], a[2,3], a[2,4], -a[1,3], -a[1,4], 0],\
             [-a[3,0], 0, a[1,0], 0, a[3,2], a[1,1] + a[3,3], a[3,4], a[1,2], 0, -a[1,4]],\
             [-a[4,0], 0, 0, a[1,0], a[4,2], a[4,3], a[1,1] + a[4,4], 0, a[1,2], a[1,3]],\
             [0, -a[3,0], a[2,0], 0, -a[3,1], a[2,1], 0, a[2,2] + a[3,3], a[3,4], -a[2,4]],\
             [0, -a[4,0], 0, a[2,0], -a[4,1], 0, a[2,1], a[4,3], a[2,2] + a[4,4], a[2,3]],\
             [0, 0, -a[4,0], a[3,0], 0, -a[4,1], a[3,1], -a[4,2], a[3,2], a[3,3] + a[4,4]]])
    return A_lift"""



# for this one the eigenvalue matches
def lift_A(a):
    to_return = [[a[0,0] + a[1,1] + a[2,2], a[2,3], a[2,4], -a[1,3], -a[1,4], 0, a[0,3], a[0,4], 0, 0],\
                 [a[3,2], a[0,0] + a[1,1] + a[3,3], a[3,4], a[1,2], 0, -a[1,4], -a[0,2], 0, a[0,4], 0],\
                 [a[4,2], a[4,3], a[0,0] + a[1,1] + a[4,4], 0, a[1,2], a[1,3], 0, -a[0,2], -a[0,3], 0],\
                 [-a[3,1], a[2,1], 0, a[0,0] + a[2,2] + a[3,3], a[3,4], -a[2,4], a[0,1], 0, 0, a[0,4]],\
                 [-a[4,1], 0, a[2,1], a[4,3], a[0,0] + a[2,2] + a[4,4], a[2,3], 0, a[0,1], 0, -a[0,3]],\
                 [0, -a[4,1], a[3,1], -a[4,2], a[3,2], a[0,0] + a[3,3] + a[4,4], 0, 0, a[0,1], a[0,2]],\
                 [a[3,0], -a[2,0], 0, a[1,0], 0, 0, a[1,1] + a[2,2] + a[3,3], a[3,4], -a[2,4], a[1,4]],\
                 [a[4,0], 0, -a[2,0], 0, a[1,0], 0, a[4,3], a[1,1] + a[2,2] + a[4,4], a[2,3], -a[1,3]],\
                 [0, a[4,0], -a[3,0], 0, 0, a[1,0], -a[4,2], a[3,2], a[1,1] + a[3,3] + a[4,4], a[1,2]],\
                 [0, 0, 0, a[4,0], -a[3,0], a[2,0], a[4,1], -a[3,1], a[2,1], a[2,2] + a[3,3] + a[4,4]]]
    return to_return

# this is the one from Taylor but eigenvalue doesn't match
def lift_A1(a):
    to_return = [[a[0,0] + a[1,1], a[1,2], a[1,3], a[1,4], -a[0,2], -a[0,3], -a[0,4], 0, 0, 0],\
              [a[2,1], a[0,0] + a[2,2], a[2,3], a[2,4], a[0,1], 0, 0, -a[0,3], -a[0,4], 0],\
              [a[3,1], a[3,2], a[0,0] + a[3,3], a[3,4], 0, a[0,1], 0, a[0,2], 0, -a[0,4]],\
              [a[4,1], a[4,2], a[4,3], a[0,0] + a[4,4], 0, 0, a[0,1], 0, a[0,2], a[0,3]],\
              [-a[2,0], a[1,0], 0, 0, a[1,1] + a[2,2], a[2,3], a[2,4], -a[1,3], -a[1,4], 0],\
              [-a[3,0], 0, a[1,0], 0, a[3,2], a[1,1] + a[3,3], a[3,4], a[1,2], 0, -a[1,4]],\
              [-a[4,0], 0, 0, a[1,0], a[4,2], a[4,3], a[1,1] + a[4,4], 0, a[1,2], a[1,3]],\
              [0, -a[3,0], a[2,0], 0, -a[3,1], a[2,1], 0, a[2,2] + a[3,3], a[3,4], -a[2,4]],\
              [0, -a[4,0], 0, a[2,0], -a[4,1], 0, a[2,1], a[4,3], a[2,2] + a[4,4], a[2,3]],\
              [0, 0, -a[4,0], a[3,0], 0, -a[4,1], a[3,1], -a[4,2], a[3,2], a[3,3] + a[4,4]]]
    return to_return

In [8]:
# define symbols
sigma, alpha = symbols('sigma, alpha') 

In [9]:
def get_e_min(A):
    #A_plus = A.copy()
    A_minus = A.copy()
    for i in range(5):
        for j in range(5):
            try:
                #A_plus[i][j] = A_plus[i][j].subs(sigma,sp)
                #A_plus[i][j] = float(A_plus[i][j])
                A_minus[i][j] = A_minus[i][j].subs(sigma,sm)
                A_minus[i][j] = float(A_minus[i][j])
            except:
                pass

    #A_plus = np.array(A_plus, dtype=float)
    A_minus = np.array(A_minus, dtype=complex)

    #e_plus,v_plus = np.linalg.eig(A_plus)
    #print(e_plus)
    #eval_p = e_plus[np.argsort(e_plus)[:3]].sum()
    #print(eval_p)

    e_minus,v_minus = np.linalg.eig(A_minus)
    #print(e_minus)
    eval_m = e_minus[np.argsort(e_minus)[:3]].sum()
    #print(eval_m)
    return eval_m

In [10]:
def solve_lifted_evans(lam, G=5/3, nu=.1):
    # define parameters
    #G = 5/3
    mu = 1
    #nu = .1
    vp = G/(G+2)
    ep = 2/((G+2)**2)
    dim = 10
    
    # define e(sigma)
    a0 = ep
    a1 = (mu*vp - 2*nu*vp + nu - (4*G*a0*mu*nu - 6*G*mu*nu*vp**2 + 8*G*mu*nu*vp - \
          2*G*mu*nu + mu**2*vp**2 - 4*mu*nu*vp**2 + 2*mu*nu*vp + 4*nu**2*vp**2 - 4*nu**2*vp + nu**2)**(1/2))/(2*G*nu)
    a2 = (a1*mu - a1*nu - 3*mu*vp/2 + mu)/(3*G*a1*nu - mu*vp + 4*nu*vp - 2*nu)
    a3 = (2*G*a2**2*nu + a2*mu - 2*a2*nu - mu/2)/(4*G*a1*nu - mu*vp + 6*nu*vp - 3*nu)
    a4 = (5*G*a2*a3*nu + a3*mu - 3*a3*nu)/(5*G*a1*nu - mu*vp + 8*nu*vp - 4*nu)
    a5 = (6*G*a2*a4*nu - 3*G*a3**2*nu + a4*mu - 4*a4*nu)/(6*G*a1*nu - mu*vp + 10*nu*vp - 5*nu)
    e_coeffs = [a0,a1,a2,a3,a4,a5]
    e = sympify(0)
    for j in range(0,len(e_coeffs)):
        e = e+e_coeffs[j]*sigma**j
    e_deriv = diff(e,sigma)
    
    
    v = sigma + vp
    vx = (v*(v-1)+G*e)/mu
    ex = v*(-.5*(v-1)**2+e)/nu
    ux = vx
    uxx = (vx*(2*v-1)+G*ex)/mu

    g = (G*e-(nu+1)*ux)/nu
    h = -ex/v
    f = 2*v - 1

    A = np.array([[0,1,0,0,0],\
                  [lam*v/nu,v/nu,v*ux/nu-uxx,lam*g,g-h],\
                  [0,0,0,lam,1],\
                  [0,0,0,0,1],\
                  [0,G,lam*v+G*ux,lam*v,f-lam]])
    
    A_lift = lift_A(A)
    
    A_lift_lim = A_lift.copy()
    A_lift = np.array(A_lift)
    for i in range(10):
        for j in range(10):
            try:
                A_lift_lim[i][j] = A_lift_lim[i][j].subs(sigma,0)
                A_lift_lim[i][j] = float(A_lift_lim[i][j])
            except:
                pass
                #A_lift_lim[i][j] = A_lift[i][j]
            #print(type(A_lift_lim[i][j]))

    A_lift_lim = np.array(A_lift_lim, dtype=complex)
    e,v = np.linalg.eig(A_lift_lim)
    which = np.argmin(e)
    e_val_ = e[which]
    e_vec_ = v[:,which]
    
    #print(np.linalg.norm(A_lift_lim @ e_vec_ - e_val_*e_vec_))
    
    # compute eigenpair for A+
    A_lim_det = -alpha*((vp/nu-alpha)*(-alpha*(-alpha*(2*vp-1-lam-alpha)-lam*vp)+\
                lam*vp*(lam+alpha))-G*(alpha*(lam*G*ep/nu+alpha*G*ep/nu))) -\
                (lam*vp/nu)*(-alpha*(-alpha*(2*vp-1-lam-alpha)-lam*vp)+lam*vp*(lam+alpha))
    A_lim_det = Poly(A_lim_det, alpha)
    
    cfs = A_lim_det.coeffs()

    if isinstance(lam, complex):
        cfs = [complex(cfs_) for cfs_ in cfs]

    e_vals = np.roots(cfs)
    e_locs = np.argsort(e_vals)[:3]
    
    e_vecs = [[G*ep*(lam+e_val)/(nu*e_val**2-vp*(lam+e_val)),\
               e_val*G*ep*(lam+e_val)/(nu*e_val**2-vp*(lam+e_val)),\
               (lam+e_val)/e_val,1,e_val] for e_val in e_vals]
    e_vecs = [[el/np.linalg.norm(np.array(e_vec)) for el in e_vec] for e_vec in e_vecs]
    
    e_vals = e_vals[e_locs].sum()
    #print(e_vals)
    e_min = get_e_min(A)
    #to_cross = [e_vecs[j] for j in e_locs]
    #e_vecs = np.cross(to_cross[0],np.cross(to_cross[1],to_cross[2]))
    #print(np.linalg.norm(A_lift_lim-A1))
    return A_lift, ux, e_val_, e_vec_, e_min

In [ ]:
"""  0.2    0.5  1.0   2.0    5.0
0.2 398.6 388.8 385.3 733.8 1755.6
0.4 211.7 182.3 175.1 325.0 762.0
0.6 222.5 123.4 111.5 198.4 449.8
2/3 226.8 114.3 100.4 175.3 391.3
0.8 236.5 103.9 85.3 142.6 307.1
1.0 253.8 100.8 73.7 113.8 229.2
1.2 274.3 106.6 69.7 98.7 183.1
1.4 300.8 117.5 70.5 91.6 154.9
1.6 347.4 131.8 74.5 89.8 138.0
1.8 397.3 148.7 80.8 91.8 128.6
2.0 450.3 167.5 88.7 96.7 124.7"""

G_vals = [.2,.4,.6,2/3,.8,1,1.2,1.4,1.6,1.8,2]
nu_vals = [0.2, 0.5, 1.0, 2.0, 5.0]

vals = np.array([[398.6, 388.8, 385.3, 733.8, 1755.6],\
                 [211.7, 182.3, 175.1, 325.0, 762.0],\
                 [222.5, 123.4, 111.5, 198.4, 449.8],\
                 [226.8, 114.3, 100.4, 175.3, 391.3],\
                 [236.5, 103.9, 85.3, 142.6, 307.1],\
                 [253.8, 100.8, 73.7, 113.8, 229.2],\
                 [274.3, 106.6, 69.7, 98.7, 183.1],\
                 [300.8, 117.5, 70.5, 91.6, 154.9],\
                 [347.4, 131.8, 74.5, 89.8, 138.0],\
                 [397.3, 148.7, 80.8, 91.8, 128.6],\
                 [450.3, 167.5, 88.7, 96.7, 124.7]])

theta_vals = np.linspace(-np.pi/2,np.pi/2,5)

for i,G_val in enumerate(G_vals):
    for j,nu_val in enumerate(nu_vals):
        G = G_val
        vp = G/(G+2)
        sp = 0
        sm = 1-vp
        
        lam_bound = vals[i][j]
        display(Markdown('# $\Lambda$ = {}'.format(lam_bound)))
        for theta_val in theta_vals:
            lam_val = lam_bound*(np.cos(theta_val) + 1j*np.sin(theta_val))

            A, u_x, e_vals, e_vecs, e_min = solve_lifted_evans(lam_val, G=G_val, nu=nu_val)
            df = e_vals + sigma*(e_min - e_vals)/sm
            
            to_disp = round(lam_val.real,4) + round(lam_val.imag,4)*1j
            display(Markdown('### $\lambda$ = {}'.format(to_disp)))
            
            sys_ = SystemSolve(10, 5, A, u_x, df, e_vecs, False, True)
            sys_.createSys()

            sigs = np.linspace(sp,sm,5)
            W_ = np.zeros((10,len(sigs)), dtype=np.complex)
            for p,sig in enumerate(sigs):
                w_val = sys_.evalW(sig)
                W_[:,p] = w_val

            rp = round(e_vals.real,4)
            ip = round(e_vals.imag,4)
            eval = rp+ip*1j

            plt.subplot(121)
            for p in range(len(W_[:,0])):
                plt.plot(sigs[:], -W_[p][:].real,label=p+1)
                plt.plot(0,-e_vecs[p].real,'*k')

            plt.suptitle('Real Parts')
            plt.ylabel(r'$Z(\sigma)$')
            plt.xticks([], [])

            plt.subplot(122)
            for p in range(len(W_[:,0])):
                plt.plot(sigs[:], -W_[p][:].imag,label=p+1)
                plt.plot(0,-e_vecs[p].imag,'*k')
            plt.xlabel(r'$\sigma$')
            plt.ylabel(r'$Z(\sigma)$')
            plt.title('Imaginary Parts')

            plt.suptitle(r'$\alpha$ = {}'.format(eval))
            plt.show()

In [ ]:
num_terms = 5

G = 2/3
nu = 0.2
lam_val = 100*1j+100

vp = G/(G+2)
sp = 0
sm = 1-vp

display(Markdown('# $\lambda$ = {}'.format(lam_val)))

w_old = np.zeros((10,1))
for num_terms in range(5,20):
    
    

    A, u_x, e_vals, e_vecs, e_min = solve_lifted_evans(lam_val,G=G,nu = nu)
    df = e_vals + sigma*(e_min - e_vals)/sm

    sys_ = SystemSolve(10, num_terms, A, u_x, df, e_vecs, False, True)
    sys_.createSys()

    w_val = np.array(sys_.evalW(sm))
    
    print('\n\n')
    print(np.linalg.norm(w_val-w_old)/np.linalg.norm(w_val))
    
    w_old = w_val

   

In [22]:
def check_conv(lam_val, G, nu, sm):
    count = 0
    start = False
    w_old = np.zeros((10,1))
    for num_terms in range(10,20,2):

        A, u_x, e_val, e_vec, e_min = solve_lifted_evans(lam_val,G=G,nu=nu)
        df = e_val + sigma*(e_min - e_val)/sm

        sys_ = SystemSolve(10, num_terms, A, u_x, df, e_vec, False, True)
        sys_.createSys()

        w_val = np.array(sys_.evalW(sm))

        check = np.linalg.norm(w_val-w_old)/np.linalg.norm(w_val)
        print(check,'\t\t',np.linalg.norm(w_val))
        
        if start:
            if check > check_old:
                count += 1
            else: 
                # this line requires three growths in a row
                count = 0
        else:
            start = True
        
        check_old = check

        w_old = w_val
        
        if count > 2:
            return False
    
    return True

In [18]:
G = 2/3
nu = 0.2

vp = G/(G+2)
sp = 0
sm = 1-vp

breaker = 0

for lam_mod in range(10,102,2):
    lam_val = lam_mod*1j
    print(lam_val,'\n\n')
    res = check_conv(lam_val, G, nu, sm)
    print(res,'\n\n\n')
    breaker = breaker + 1 - res
    
    if breaker > 3:
        break
    #print(check_conv(lam_val, G, nu, sm),'\n\n\n')
    

10j 


3.1622776601683795
0.3030015439071864
0.11676432345321454
0.05924244222920787
0.03622376977887242
0.030055443585256025
0.029499172604919643
0.030234339460918715
True 



12j 


3.1622776601683786
0.3901635966177822
0.13108600146332752
0.057430708796912144
0.03140793125232394
0.02826176303279909
0.03015067315937884
0.031943510796024255
True 



14j 


3.162277660168379
0.4646779092544549
0.13968461613372138
0.05099116887537949
0.023645700352136274
0.0266513104599537
0.03130661669337165
0.03388135514670406
False 



16j 


3.1622776601683795
0.5185990108382766
0.14521860928255106
0.04258173319290565
0.01660668408901999
0.02773767532443541
0.03382504318421711
0.03635724933035536
False 



18j 


3.1622776601683795
0.5549687129482659
0.1513366499940384
0.03709471917799986
0.0191333294944876
0.03281934710064036
0.037881965127176113
0.03936390917114957
False 



20j 


3.16227766016838
0.5814479176203581
0.16149974458820412
0.041526912311059344
0.03201500950981229
0.04107723160515101


In [18]:
G = 2/3
nu = 0.2

vp = G/(G+2)
sp = 0
sm = 1-vp

breaker = 0

for lam_mod in range(10,101,10):
    lam_val = lam_mod*1j
    print(lam_val,'\n\n')
    res = check_conv(lam_val, G, nu, sm)
    print(res,'\n\n\n')
    breaker = breaker + 1 - res
    
    if breaker > 3:
        break

10j 


3.162277660168379 		 4.0574826132772985
0.03133563618268347 		 4.1390438883469605
True 



20j 


3.1622776601683795 		 8.006712197465937
0.038625451127024174 		 8.26233979577012
True 



30j 


3.16227766016838 		 13.444409363342048
0.04281833044536485 		 13.789443114519148
True 



40j 


3.1622776601683795 		 19.596435777185995
0.07413296361908417 		 20.603808192399182
True 



50j 


3.16227766016838 		 27.625417638617584
0.08635100583974932 		 30.079211046824312
True 



60j 


3.162277660168379 		 41.5566706713178
0.05830199669404735 		 43.90547054921921
True 



70j 


3.16227766016838 		 61.26601249077711
0.02636752841564068 		 60.1921580971495
True 



80j 


3.1622776601683795 		 80.5634542412963
0.11234304986279649 		 74.67779156399321
True 



90j 


3.162277660168379 		 90.93639129943296
0.24927412581682637 		 86.39275017393025
True 



100j 


3.162277660168379 		 87.40319994509797
0.389005293872695 		 105.69028509571652
True 





In [ ]:
G = 2/3
nu = 0.2

vp = G/(G+2)
sp = 0
sm = 1-vp

breaker = 0

for lam_mod in range(150,210,10):
    lam_val = lam_mod*1j
    print(lam_val,'\n\n')
    res = check_conv(lam_val, G, nu, sm)
    print(res,'\n\n\n')
    breaker = breaker + 1 - res
    
    if breaker > 3:
        break
    #print(check_conv(lam_val, G, nu, sm),'\n\n\n')

150j 


3.162277660168379 		 1461.1429630467335
2.7651045893537 		 657.5017206830455
2.0006526184552365 		 588.8408275521691
0.9243602053096904 		 706.0105909698796
0.6022980669087921 		 552.0787299973197
True 



160j 


3.162277660168379 		 1910.7795264994654
2.79757627590186 		 931.0068019564059
1.709059363509755 		 1014.7769259812694
0.996513316242745 		 968.0519940679418
0.8561054813301443 		 599.8689138071798
True 



170j 


3.16227766016838 		 2453.8975123521996
2.610477442019539 		 1401.544447860684
1.5910963798042534 		 1571.4391671035526
1.1421953813746124 		 1240.07502846215
1.4058351479620983 		 563.6103752588855
True 



180j 


3.162277660168379 		 3113.7952591377702
2.3837779882053156 		 2117.7160986788303
1.558789176791159 		 2272.474520245838
1.371544290120647 		 1505.5853697003488
2.869431621271938 		 419.6925131065919
True 



190j 


3.1622776601683795 		 3922.830575974881
2.1924632186167514 		 3125.77500102132
1.578034678709893 		 3133.2897862177774
1.692102363948

In [ ]:
G = 2/3
nu = 0.2

vp = G/(G+2)
sp = 0
sm = 1-vp

breaker = 0

for lam_mod in range(226,228,2):
    lam_val = lam_mod*1j
    print(lam_val,'\n\n')
    res = check_conv(lam_val, G, nu, sm)
    print(res,'\n\n\n')
    breaker = breaker + 1 - res
    
    if breaker > 3:
        break
    #print(check_conv(lam_val, G, nu, sm),'\n\n\n')